#CNN Mimarilerinn Karşılaştırılması

## Gerekli Kütüphanelerin yüklenmesi

In [ ]:
# Google Colab'da çalıştırmak için gereken kütüphaneleri yükleyin
!pip install tensorflow keras matplotlib

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

## Veri setinin yüklenmesi

In [ ]:
# CIFAR-10 veri setini yükleyin
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

input_shape = (32, 32, 3)  # CIFAR-10 için giriş boyutu
num_classes = 10

## Normalizasyon

In [ ]:
# Verileri normalize edin ve etiketleri one-hot formatına dönüştürün
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Eğitim ve test verilerini artırma
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

# Metrikleri saklamak için boş bir liste oluşturun
model_names = []
accuracies = []

#Modellerin Oluşturulması

## Lenet

In [ ]:
# Model tanımlama ve eğitim fonksiyonları
def create_lenet5():
    model = Sequential([
        Conv2D(6, (5, 5), activation='tanh', padding='same', input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, (5, 5), activation='tanh', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(120, (5, 5), activation='tanh', padding='valid'),
        Flatten(),
        Dense(84, activation='tanh'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Alexnet

In [ ]:
def create_alexnet():
    model = models.Sequential()

    # 1. Konvolüsyonel Katman
    model.add(layers.Conv2D(96, (5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # 2. Konvolüsyonel Katman
    model.add(layers.Conv2D(256, (5, 5), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # 3. Konvolüsyonel Katman
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # 4. Konvolüsyonel Katman
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # 5. Konvolüsyonel Katman
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Düzleştirme Katmanı
    model.add(layers.Flatten())

    # Tam Bağlantılı Katmanlar
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## VGG

In [ ]:
def create_vgg16():
    base_model = VGG16(weights=None, input_shape=(32, 32, 3), classes=10)
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return base_model

## Resnet

In [ ]:
def create_resnet():
    base_model = ResNet50(weights=None, input_shape=(32, 32, 3), classes=10)
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return base_model

## Densnet

In [ ]:
def create_densenet():
    base_model = DenseNet121(weights=None, input_shape=(32, 32, 3), classes=10)
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return base_model

## Efficentnet

In [ ]:
def create_efficientnet():
    base_model = EfficientNetB0(weights=None, input_shape=(32, 32, 3), classes=10)
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return base_model

## Modelleri listeleme

In [ ]:
# Model listesi ve isimler
models = [
    (create_lenet5(), 'LeNet-5'),
    (create_alexnet(), 'AlexNet'),
    (create_vgg16(), 'VGG16'),
    (create_resnet(), 'ResNet50'),
    (create_densenet(), 'DenseNet121'),
    (create_efficientnet(), 'EfficientNetB0')
]

## Modellerin eğitimi ve doğrulama

In [ ]:
# Her modeli eğitin ve doğruluğunu hesaplayın
for model, name in models:
    print(f"Training {name}...")
    history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                        validation_data=(x_test, y_test),
                        epochs=10, verbose=1)
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    model_names.append(name)
    accuracies.append(accuracy)
    print(f"{name} Test Accuracy: {accuracy:.4f}")

## Modellerin görselleştirilmesi

In [ ]:
# Sonuçları görselleştirin
plt.figure(figsize=(12, 6))
plt.bar(model_names, accuracies, color='skyblue')
plt.xlabel('Model')
plt.ylabel('Doğruluk')
plt.title('CNN Mimarileri ve CIFAR-10 Doğruluk Karşılaştırması')
plt.show()